In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import math

In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # 维度必须是偶数
        if dim % 2 != 0:
            raise ValueError(f"Cannot use sin/cos positional encoding with odd dim (got dim={dim})")

        # 构建一个长为max_len 列为dim的二维向量
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2, dtype=float) * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position/div_term)
        pe[:, 1::2] = torch.cos(position/div_term)
        # 增加一个维度用于广播
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)
        self.drop_out = nn.Dropout(dropout)
        self.dim = dim

    def forward(self, emb):
        emb = emb + pe[:, emb.size(0)]
        return self.drop_out(emb)

In [3]:
def self_attention(query, key, value, dropout=None, mask=None):
    # query最后一维是维度信息
    d_k = query.size(-1)
    # query * key的转置 再除以维度开根号
    scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        mask.cuda()
        scores = scores.masked_fill(mask == 0, -1e9)
    # 最后再套上一层softmax
    self_atten = F.softmax(scores, dim=-1)
    if dropout is not None:
        self_atten = dropout(self_atten)
    return 

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()

    def forward(self, head, d_model, query, key, value, dropout=0.1, mask=None):
        assert (d_model % head == 0)
        self.head_num = d_model // head
        self.head = head
        self.d_model = d_model

        self.linear_query = nn.Linear(d_model, d_model)
        self.linear_key = nn.Linear(d_model, d_model)
        self.linear_value = nn.Linear(d_model, d_model)

        self.linear_out = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.atten = None

        n_batch = query.size(0)

        query = self.linear_query(query).view(n_batch, -1, self.head, self.head_num).transpose(1, 2)
        key = self.linear_query(key).view(n_batch, -1, self.head, self.head_num).transpose(1, 2)
        value = self.linear_query(value).view(n_batch, -1, self.head, self.head_num).transpose(1, 2)

        x, self_attn = self_attention(query, key, value, dropout, mask)
        
        x = x.transpose(1, 2).contiguous().view(n_batch, -1, self.head_num * self.head)

        return self.linear_out(x)

In [18]:
class LayerNorm(nn.Module):

    def __init__(self, feature, eps=1e-6):
        """
        :param feature: self-attention 的 x 的大小
        :param eps:
        """
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(feature))
        self.b_2 = nn.Parameter(torch.zeros(feature))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [19]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout=0.1):
        super(SublayerConnection, self).__init__()
        # layernorm first
        self.layer_norm = LayerNorm(size)
        # do dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return self.dropout(self.layer_norm(x + sublayer(x)))

In [20]:
# class PositionWiseFeedForward(nn.Module):
#     """
#     FFN(x)=max(0,xW1+b1)W2+b2
#     """
#     def __init__(self, d_model, d_ff, dropout=0.1):
#         super(PositionWiseFeedForward, self).__init__()
#         self.w_1 = nn.Linear(d_model, d_ff)
#         self.w_2 = nn.Linear(d_ff, d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         return self.w_2(self.dropout(self.w_1(x).relu()))

In [22]:
# FFN
# class PositionWiseFeedForward(nn.Module):
#     """
#     w2(relu(w1(layer_norm(x))+b1))+b2
#     """
#     def __init__(self, d_model, d_ff, dropout=0.1):
#         super(PositionWiseFeedForward, self).__init__()
#         self.w_1 = nn.Linear(d_model, d_ff)
#         self.w_2 = nn.Linear(d_ff, d_model)
#         self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
#         self.dropout_1 = nn.Dropout(dropout)
#         self.relu = nn.ReLU()
#         self.dropout_2 = nn.Dropout(dropout)

#     def forward(self, x):
#         inter = self.dropout_1(self.relu(self.w_1(self.layer_norm(x))))
#         output = self.dropout_2(self.w_2(inter))
#         return output

class PositionWiseFFN(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionWiseFFN, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.relu(self.w_1(x)))

In [32]:
def clones(module, n):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(n)])

class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.embed = self.embedding

    def forward(self, x):
        # 输入乘以维度开方，防止位置编码过大把原始数据影响了
        return self.embed(x) * math.sqrt(self.d_model) 

In [33]:
class EncoderLayer(nn.Module):
    def __init__(self, size, attn, feed_forward, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.attn = attn
        self.feed_forward = feed_forward
        self.sublayer_connection = clones(SublayerConnection(size, dropout), 2)

    def forward(self, x, mask):
        x = self.sublayer_connection[0](x, lambda x: self.attn(x, x, x, mask))
        return self.sublayer_connection[1](x, self.feed_forward)

In [34]:
class Encoder(nn.Module):
    def __init__(self, n, encoder_layer):
        super(Decoder, self).__init__()
        self.encoder_layer = clones(encoder_layer, n)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, src_mask):
        for layer in self.encoder_layer:
            x = layer(x, src_mask)
        return self.norm(x)

In [35]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer_connection = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer_connection[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer_connection[1](x, lambda x: self.self_attn(x, m, m, src_mask))
        return self.sublayer_connection[2](x, self.feed_forward)

In [36]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)